In [89]:
include("../Algebra/Gradients.jl")
include("../Algebra/Hamiltonians.jl")
include("../Algebra/Matrices.jl")
include("../Algebra/Propagators.jl")
include("../Amplitudes/Chebyshev.jl")
include("../Costs/Costs.jl")
include("../Gates/Xgate.jl")

U2 (generic function with 1 method)

In [90]:
using LinearAlgebra, Optim, Plots
using BenchmarkTools, Distributions

In [91]:
ω_c = 5
ω_q = 2
χ = 3e-3
N = 6
amp_c = 1
amp_q = 1

1

In [92]:
I_qubit = Matrix(I,2,2)
I_cavity = Matrix(I,N,N)
X_gate = kron(x_gate(N, Array[[2,5]]), I_qubit)

#X_gate = kron(x_gate(N, Array[[2,3], [4,5]]), I_qubit)

θ = π/2.5
RZ = kron(Rz(N, Array[[3,4]], θ/2), I_qubit)
RX = kron(Rx(N, Array[[3,4]], θ/4), I_qubit)
Udue = kron(U2(N, Array[[3,4]], [θ/2 θ/3 θ/4] ), I_qubit)

sx = kron(I_cavity, [0 1; 1 0])
sy = kron(I_cavity, [0 -1im; 1im 0])

ψ_initial = rand(N)
ψ_initial = kron(ψ_initial/norm(ψ_initial), [0,1])

12-element Vector{Float64}:
 0.0
 0.15714193695644985
 0.0
 0.5218568024193412
 0.0
 0.36886796764867485
 0.0
 0.6761114807102022
 0.0
 0.13986602658713793
 0.0
 0.30036489828133367

In [93]:
ψ_final_u = Udue * ψ_initial
ψ_final_x = RX * ψ_initial
ψ_final_x_gate = X_gate * ψ_initial

12-element Vector{Float64}:
 0.0
 0.15714193695644985
 0.0
 0.13986602658713793
 0.0
 0.36886796764867485
 0.0
 0.6761114807102022
 0.0
 0.5218568024193412
 0.0
 0.30036489828133367

In [94]:
# generating matrices
a,adag,sp,sm,sz = generate_matrices(N)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0 1 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 1; 0 0 … 0 0], [0 0 … 0 0; 1 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 1 0], [1 0 … 0 0; 0 -1 … 0 0; … ; 0 0 … 1 0; 0 0 … 0 -1])

In [81]:
disp(α) = exp((α * adag - α' * a))
ECD(β) = disp(β / 2) * sp + disp(- β / 2) * sm
ROT(ϕ, θ) = exp(-1im * θ / 2 * ( sx * cos(ϕ) + sy * sin(ϕ) )) 
snap(x) = kron(Diagonal(exp.(-1im.*x)),I_qubit)

snap (generic function with 1 method)

In [82]:
function cost(coefficients, unitary, N)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)
    end

    return 1 - norm(tr(propagator*unitary') / N)^2
    
end

function cost_ecd_rot_ecd(coefficients, unitary, N)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ECD(β_i[i] + β_r[i] * 1im) * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)'
    end

    return 1 - norm(tr(propagator*unitary') / N)^2
    
end

function gradient(coefficients, unitary, N, h, cost_before_increment)

    L = length(coefficients)
    gradient = zeros(L)

    for i in 1:L
        coeffs_copy = copy(coefficients)
        coeffs_copy[i] = coefficients[i] + h
        gradient[i] = (cost(coeffs_copy, unitary, N) - cost_before_increment) / h
    end

    return gradient

end

function gate_composition(coefficients, N, l_coeff)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)
    end

    return propagator
    
end

function cost_state_prep(coefficients, N, ψ_n, ψ_m)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)
    end

    return 1 - norm(ψ_m' * propagator * ψ_n)^2
    
end


function gradient_state_prep(coefficients, N, h, cost_before_increment, ψ_n, ψ_m)

    L = length(coefficients)
    gradient = zeros(L)

    for i in 1:L
        coeffs_copy = copy(coefficients)
        coeffs_copy[i] = coefficients[i] + h
        gradient[i] = (cost_state_prep(coeffs_copy, N, ψ_n, ψ_m) - cost_before_increment) / h
    end

    return gradient

end

gradient_state_prep (generic function with 1 method)

In [83]:
snapgate = snap(rand(N))

12×12 Matrix{ComplexF64}:
 0.849522-0.527553im       0.0-0.0im       …       0.0+0.0im
      0.0-0.0im       0.849522-0.527553im          0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im       …       0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im       …       0.0-0.0im
      0.0+0.0im            0.0+0.0im          0.884872-0.465834im

In [84]:
f(x) = cost(x, RX, N * 2)
g(x) = cost_state_prep(x, N * 2, ψ_initial, ψ_final_x_gate)

g (generic function with 1 method)

In [85]:
costs = []
for depth in 18:6:36
    it_costs = []
    for iteration in 1:1
        println("blocks: ", depth," iterations :", iteration)
        l_coeff = depth * 4
        coefficients = zeros(l_coeff)
        # coefficients[1:Int(l_coeff / 2)] = [opt_coefficients[1:Int(length(opt_coefficients)/2)]..., (rand(Int(l_coeff / 2 - length(opt_coefficients)/2))*2 .-1)...]
        # coefficients[Int(l_coeff / 2) + 1:end] = [opt_coefficients[Int(length(opt_coefficients)/2)+1:end]..., zeros(Int(l_coeff / 2 - length(opt_coefficients)/2))...]
        coefficients[1:Int(l_coeff / 2)] = rand(Int(l_coeff / 2))*2 .-1
        coefficients[Int(l_coeff / 2) + 1:end] = rand(Int(l_coeff / 2)) * 2 * pi .- pi
        res = Optim.optimize(f, coefficients , LBFGS(), Optim.Options(show_trace=true, show_every=100, iterations = 1000))
        push!(it_costs,Optim.minimum(res))
        println(Optim.minimum(res))
        println(Optim.minimizer(res))
    end
    println(minimum(it_costs))
    push!(costs,minimum(it_costs))
end

blocks: 18 iterations :1
Iter     Function value   Gradient norm 


     0     9.932570e-01     2.600554e-02
 * time: 0.00012111663818359375


   100     1.174773e-01     1.032065e-02
 * time: 44.95827889442444


   200     7.666136e-02     5.862265e-03
 * time: 90.7849760055542


   300     6.319234e-02     1.856986e-03
 * time: 137.39284992218018


...

   900     3.807860e-03     1.529107e-03
 * time: 1165.7656180858612


  1000     3.505178e-03     1.555439e-03
 * time: 1292.391347169876


0.0035051783341304432
[-1.0518688076327998, -0.1460783191155432, -0.7543686246640113, 2.8029578047626247, 1.8958718569413135, -0.6197523552287705, 0.13556664646986832, 0.8229533901815008, -0.8039836258044708, 0.24149347713344854, -0.6296851040954266, -0.9835922758859338, 0.4971615179273996, 0.5247127535179286, 0.49667595382222773, 0.36128253007622174, -0.6402390989731267, 0.2150844880913754, 0.6120588464552729, 0.16837538626219828, -0.7148573797587545, -0.22321914074402133, 0.29351906417840484, 1.5919306251389849, 0.10284642885937424, -0.44877681083426674, 1.0484341506828263, 1.1640011248507671, 0.7350844905765151, 2.121432451343145, 1.2232266767462625, -0.7322517427586578, 1.0049529941018447, -0.008964763752227793, 0.6186633358149567, 1.3108514221059753, 0.8207690869288334, 0.2013754433527983, -0.0963038568422146, 0.101235578282585, 0.7517339040360522, 0.31208980981494283, -1.549622276550702, 1.1282468227943054, -0.5331009780728758, 1.1720599277036925, 0.10583547866283147, 0.388813576

Iter     Function value   Gradient norm 
     0     9.906320e-01     3.970399e-02
 * time: 0.00017690658569335938


InterruptException: InterruptException:

In [86]:
RX = kron(Rx(N, Array[[3,4]], θ*2.5), I_qubit)

12×12 Matrix{ComplexF64}:
 1.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  1.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     1.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  1.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  1.0+0.0im

In [87]:
f(x) = cost(x, RX, N * 2)

f (generic function with 1 method)

In [88]:
costs = []
for depth in 18:6:36
    it_costs = []
    for iteration in 1:1
        println("blocks: ", depth," iterations :", iteration)
        l_coeff = depth * 4
        coefficients = zeros(l_coeff)
        # coefficients[1:Int(l_coeff / 2)] = [opt_coefficients[1:Int(length(opt_coefficients)/2)]..., (rand(Int(l_coeff / 2 - length(opt_coefficients)/2))*2 .-1)...]
        # coefficients[Int(l_coeff / 2) + 1:end] = [opt_coefficients[Int(length(opt_coefficients)/2)+1:end]..., zeros(Int(l_coeff / 2 - length(opt_coefficients)/2))...]
        coefficients[1:Int(l_coeff / 2)] = rand(Int(l_coeff / 2))*2 .-1
        coefficients[Int(l_coeff / 2) + 1:end] = rand(Int(l_coeff / 2)) * 2 * pi .- pi
        res = Optim.optimize(f, coefficients , LBFGS(), Optim.Options(show_trace=true, show_every=100, iterations = 1000))
        push!(it_costs,Optim.minimum(res))
        println(Optim.minimum(res))
        println(Optim.minimizer(res))
    end
    println(minimum(it_costs))
    push!(costs,minimum(it_costs))
end

blocks: 18 iterations :1
Iter     Function value   Gradient norm 


     0     9.956750e-01     2.093773e-02
 * time: 0.00014090538024902344


InterruptException: InterruptException:

# stoc gradient descent

In [11]:
costs_final = []
for depth in 64:8:64
    it_costs = []
    for iteration in 1:3
        l_coeff = depth * 4
        coefficients = zeros(l_coeff)
        coefficients[1:Int(l_coeff / 2)] = rand(Int(l_coeff / 2))*2 .-1
        coefficients[Int(l_coeff / 2) + 1:end] = rand(Int(l_coeff / 2)) * 2 * pi .- pi
        coefficients = [0.8690408632149678, -0.7236416401971805, -0.0908719850588177, 0.8258628776740167, 0.9056445769487957, -0.8220939210081029, 0.29601861853616424, 0.7813552229187792, 0.9837594907380149, -0.10686709645389225, -0.6656407250459772, 0.992278727387623, -0.4893461915705191, -1.093993962903741, 0.4845407107960842, -0.33857113975897773, -0.438797994554033, -0.5977365084149797, 0.20827466739249778, -0.8959109454265681, 0.5383716212207733, 1.3219169845803183, 0.8578064725920909, -0.7256810259845291, 0.37256259960242327, -1.2114316020995235, -0.282902962907945, -0.26590734759323503, 0.5548879566071089, 0.6803304120122661, -1.0549066677248888, 0.4013045273205048, 0.039560949181603355, 1.0529793828938236, -0.044290684794351255, -0.21790564460826214, 0.7641852601374161, 0.9310403877890154, -0.24519813978115026, 0.6528372052655407, -1.2521949229101696, -0.21682088705693697, 0.10911919627652335, -0.31451889968452584, 0.07428438971871276, 0.8168118972903352, -0.4306959382326172, -1.31201487771103, -0.5551956081923786, -0.24938592347753555, 0.7261072879386111, 0.3449918944429117, 1.0218022420859423, 0.9586632779953215, 0.9830015979456759, 1.1800344827793208, -0.6015729201693757, 0.703237921035128, -0.8659071933834304, 0.7405471254766851, 0.5502125342538493, -0.1279261558044507, 0.5436405284887846, 0.7561715320031175, -1.8294742583884145, 0.8274849373492085, -0.5023928927013781, -0.2106775708083146, -0.25623844263590434, -0.13700323024471564, -0.5427032401830472, 0.8481733035397572, 2.857123893083541, 0.6722772567617452, 3.3393904892971857, -2.7921664352181588, -1.4927954172559996, 0.3500341652427946, 2.178592973947075, 2.0048971146113073, -1.3350642362244973, 0.9711032282505817, 2.3816703973089663, -2.6138104747765247, 2.5134841942159674, -1.9047337097507626, 2.2260526473073146, -1.8796539188004344, 1.6517968966982153, 1.4620145034237324, 1.8366376463164558, -1.3614199467978896, -1.3067881525922014, -0.8271088962045482, -1.3523594955345797, -0.8216607978872826, -1.015386302411769, -1.4951025575413324, 2.199130944918348, -2.098541424724332, 1.8106459325712736, -2.7528680557326783, 1.434522172097638, 0.23235089810469334, -1.3427653914596476, -2.1865004319117882, -4.499311913607152, 3.5233665940145906, 0.11147763894762594, 1.7253123134995119, 0.3069539608771961, 0.7456729443236856, 0.9675539799184902, -2.0807423878078692, -0.21274877285924562, -2.4625967782327733, -0.8958201449217154, -0.16506248177386307, -0.7781344488152088, 0.5657089824636788, 0.49103015856219095, -1.893089322888448, 1.8586226474868552, 1.3367905090651682, -0.845322993521433, -2.414547922619251, -2.0653040809018646, 2.13700608386267, 0.6560460356595541, 1.2398223292845443, 2.962599533837159, 1.1325322497723143, 3.463842619105753, -3.938053691752728, -0.9569023982656585, 1.605238908260515, 0.6974820952130029, 2.5406564651431, 2.285077612710168, 0.027961409188739638, 1.0736422395589698, 2.4870006619787572, 0.692378916991639, -2.718696870022777]
        beta1 = 0.8
        beta2 = 0.999
        alpha = 0.01
        m = zeros(length(coefficients))
        v = zeros(length(coefficients))
        coeffs_stored = []
        costs = []
        cost_prev_it = cost(coefficients, RX, N*2)  #cost_state_prep(coefficients, N * 2, ψ_initial, ψ_final_x_gate) #

        for i in 1:1000
            grad = gradient(coefficients, RX, N * 2, 1e-8, cost_prev_it) #gradient_state_prep(coefficients, N*2, 1e-8, cost_prev_it, ψ_initial, ψ_final_x_gate) #
            for n in 1:length(coefficients)
                m[n] = beta1 * m[n] + ( 1 - beta1 ) * grad[n]
                v[n] = beta2 * v[n] + ( 1 - beta2 ) * grad[n]^2
                m_norm = m[n] / ( 1 - beta1^i )
                v_norm = v[n] / ( 1 - beta2^i )
                coefficients[n] = coefficients[n] - alpha * m_norm / (sqrt(v_norm) + 1e-8)
            end
            cost_prev_it = cost(coefficients, RX, N*2)# cost_state_prep(coefficients, N * 2, ψ_initial, ψ_final_x_gate) #
            push!(costs, cost_prev_it)

            if i % 50 == 0
                if norm(grad) < 1e-6
                    break
                end
                println(i)
                println(cost_prev_it)
            end
        end
        println("finished: d = ", depth)
        println(costs[end])
        push!(it_costs,costs[end])
        println(coefficients)
    end
    println(it_costs)
    println(minimum(it_costs))
    push!(costs_final, minimum(it_costs))
end
costs_final

50
0.1141917226386221


100
0.03329383329160829


150
0.010720497771225856


200
0.006440674105218869


250
0.0049437727392727115


...


0.0014801301144540258
950


0.00143687628948308
1000


0.0015569004887809879
finished: d = 64
0.0015569004887809879
[0.9220184640273602, -1.1641243240602506, 0.4048778825453255, 0.5998563414243495, 1.4193162995319635, -0.36088118906179323, 0.5508300066366707, 0.8015956210944428, 1.2028585710600197, 0.19084010106064914, -0.9731975661841441, 0.9135433057820816, -0.6611637699571956, -1.0345370398278404, 0.6079808853769756, -0.40239888684631464, -0.4653759041220447, -0.3808347148641949, 0.3952921377997406, -0.7740552059868105, 0.6766266849802979, 1.473063226256188, 0.7477866153987138, -0.21906139467202848, 0.5576913806183862, -1.6133074675332602, 0.26737767946687885, -0.009585865705758756, 0.3239665921698326, 0.48598927161649697, -1.5903539532429976, 0.3381501089619145, -0.4401862180726339, 1.7927968558157037, 0.009592449491889012, -0.041002778759054635, 0.660201634188688, 1.2271826571201523, -0.5064648700153209, 0.8759669682544846, -1.0739055822743246, -0.28176944825520434, 0.65209978561725, -0.6355715139562277, 0.2774989440777268, 1.3498437

[0.0015569004887809879, 0.0015569004887809879, 0.0015569004887809879]
0.0015569004887809879


1-element Vector{Any}:
 0.0015569004887809879

In [21]:
coeffs = [0.8328194938452032, -0.8105522750655366, -0.03947596969773574, 0.8150417127415052, 0.9355586497841004, -0.819672727969885, 0.15908257377596594, 0.8969730569787527, 1.0867987077197083, -0.12382796442095499, -0.6769159418813943, 1.0246426230169303, -0.5084405635279222, -1.0268972961310772, 0.42484396266977487, -0.30984213118061454, -0.44195548463552226, -0.6327731382009563, 0.2470520241498077, -0.9046312383937988, 0.5670153531055953, 1.3215582797100167, 0.8729333306332443, -0.78886152481934, 0.3445569548035575, -1.2206696780026396, -0.25471100801485647, -0.3585072716256307, 0.6870581315744397, 0.6545040107617951, -1.0221685943730465, 0.3343131234446875, -0.08486885305074147, 1.16534326489163, -0.08178072747870009, -0.17367128231750495, 0.7301925883287174, 0.7960381150713036, -0.2445176384048139, 0.7450024213033812, -1.2896520166163863, -0.2511261526654848, 0.18899633152030276, -0.3134353271761458, 0.040134442582384806, 0.7718476862874823, -0.4457105371804348, -1.2658170397923836, -0.578778138744985, -0.27405676265384593, 0.6995176637177795, 0.35946077517978764, 1.0020099923271824, 0.9306624492297949, 0.9741238940562686, 1.1608927613144042, -0.5310290099006071, 0.8115833948525197, -0.7784531065547783, 0.7352221959115539, 0.5417078187690723, -0.21113457049160275, 0.5103730707595793, 0.7551026732286986, -1.7846390041521236, 0.8631165439769484, -0.5498701334703305, -0.26861474853829703, -0.27455529220647396, -0.07688130726216859, -0.5631568455670428, 0.8533900226411945, 2.8067345186309858, 0.6785936932829509, 3.3943749838043784, -2.805214370969844, -1.4757375492498623, 0.29692768551759546, 2.281415636662691, 2.160137328769026, -1.307321749251418, 1.033561682283, 2.3601394944015057, -2.696457234103049, 2.4969210954848147, -1.906381025598878, 2.2434123597180444, -1.9770399537034882, 1.690842335222719, 1.535746792078091, 1.8604714721583955, -1.3743895036802214, -1.291663669776218, -0.762668513556034, -1.3646714887473783, -0.7879082846775004, -0.9459496839509586, -1.5081193660443275, 2.163188081396188, -2.0477877934412, 1.8200709265036517, -2.7617245086967235, 1.4249304092894297, 0.24880088065636674, -1.2535266524595052, -2.0585026020721804, -4.576841695056939, 3.5387398533822982, -0.07531806514326735, 1.8453317273419196, 0.42767796713041867, 0.7150419102279323, 0.7474552662694737, -1.6860907528015776, -0.2962280834604321, -2.386788332449464, -1.1771757193252919, 0.09208258390797398, -0.8624876874455844, 0.5131833036461091, 0.6136063613800034, -2.0172431784872105, 1.9575955203123583, 1.3468300140519078, -0.7787319428945259, -2.5374797856832374, -1.9401420236852418, 2.0676338346841168, 0.7064923766586017, 1.1338034698609585, 2.9969098154086633, 1.0459448874008392, 3.4682585115304194, -3.9796395422002675, -0.8624504151631956, 1.4625786737059918, 0.8235301000679373, 2.5250726036878977, 2.085654990962269, -0.17147063484675754, 0.7961241933337927, 2.6289885365160464, 0.6582125941103614, -2.531971259434754]
f(x) = cost(x, X_gate, N*2)
println(f(coeffs))

opt_perturbed = ComplexF64[]

for coeff in coeffs
    perturbed = coeff + (rand()*0.002-0.001)
    push!(opt_perturbed,perturbed)
end

f(opt_perturbed)

0.0003768396576976363


0.0004272672126374166